In [1]:
import pandas as pd
import numpy as np

# 지도에 표시
import folium
from folium.features import DivIcon
from folium import plugins

In [2]:
file_name = "data/data4.txt"
time = '30S'

In [3]:
def read_file(name):

    df = pd.read_csv(name, sep='\s+', header=None)
    df.drop(df.index[1:4], inplace=True)
    df = df.rename(columns=df.loc[0, :])
    df.drop(df.index[0], inplace=True)

    temp = df[['Latitude', 'Longitude', 'Time']].astype({
            'Latitude': np.float,
            'Longitude': np.float
    }).dropna()
    temp.Time = pd.to_datetime(temp['Time'])
    
    return temp

In [14]:
def tracking(name, interval):
    
    temp = read_file(name)
    
    df_sec = temp
    df_sec.index = temp.Time
    df_sec = df_sec.dropna()
    
    df = df_sec.resample(interval).mean()
    df.dropna(inplace=True)
    
    m = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()], zoom_start=100)
    
    for loc in df.values.tolist():
        folium.CircleMarker(loc, radius=5, 
                                color='green',).add_to(m)

    return m

In [5]:
def tracking_num(name, interval):
    
    temp = read_file(name)
    
    df_sec = temp
    df_sec.index = temp.Time
    df_sec = df_sec.dropna()
    
    df = df_sec.resample(interval).mean()
    df.dropna(inplace=True)
    
    m = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()], zoom_start=100)
    
    for i in range(0, len(df)):

        name = '<div style="font-size: 18pt; color : black">%s</div>' %str(i)
        p = [df.iloc[i].Latitude, df.iloc[i].Longitude]
        folium.Marker(p, icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html=name
                )).add_to(m)
        m.add_child(folium.CircleMarker(p, radius=10))
        
        
    return m

In [6]:
def create_geojson_features(data):
    features = []
    for _, row in data.iterrows():

        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point',
                'coordinates':[row['Longitude'],row['Latitude']]
            },
            'properties': {
                'time': row["Time"],
                'style': {'color': "crimson"},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': "crimson",
                    'fillOpacity': 0.05,
                    'stroke': 'true',
                    'radius': 5
                }
            }
        }     
        features.append(feature)

    return features

def make_map(features, data, interval):   
    p = 'PT'+interval
    myMap = folium.Map(location=[data.Latitude.mean(), data.Longitude.mean()], tiles='cartodbpositron', zoom_start=100)
    plugins.TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period=p
        , add_last_point=True
        , transition_time=200
        , auto_play=True
        , max_speed=5
        , loop=True
        , loop_button=True
        , time_slider_drag_update=True
    ).add_to(myMap)
  
    return myMap

In [7]:
def popup(name, interval):
    
    data = read_file(name)
    
    data.index = data.Time
    data = data.resample(interval).mean().reset_index()
    
    data.Time = data.Time.astype(str)
    
    for i in range(0, len(data)):
        a = data.iloc[i, 0].split(' ')
        data.iloc[i, 0] = a[0] + 'T' + a[1]  

    
    features = create_geojson_features(data)
    myMap = make_map(features, data, interval)
    
    return myMap

In [8]:
def makeline(name, interval):
    temp = read_file(name)
    df_sec = temp
    df_sec.index = temp.Time
    df_sec = df_sec.dropna()
    
    df = df_sec.resample(interval).mean()
    df.dropna(inplace=True)
    
    m = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()], zoom_start=100, 
                  fill_color='blue', number_of_sides=3)
    my_PolyLine=folium.PolyLine(locations=df.values,weight=1)
    m.add_children(my_PolyLine)
    return m

In [15]:
m = tracking(file_name, time)
m

In [16]:
phone = pd.read_csv("data/phone gps.txt", sep=' ', header=None)

In [18]:
for loc in phone.values.tolist():
        folium.CircleMarker(loc, radius=3, 
                            fill_color='blue', number_of_sides=3).add_to(m)

In [19]:
m

In [10]:
tracking_num(file_name, time)

In [12]:
popup(file_name, '15S') # 15S~

# folium.map.save() 
popup(file_name, '15S').save("data4.html")

In [20]:
# folium.map.save() 
m.save("data4_2.html")

In [44]:
makeline(file_name, time)

C:\Users\lonj7\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  del sys.path[0]
